In [9]:
import pandas as pd
s= {'text':sentences,'labels':1}
s2={'text':random_sentences,'labels':0}
df1=pd.DataFrame(s2)
df2=pd.DataFrame(s)
df=pd.concat((df1,df2))
df= df.sample(frac=1)
df.head()



,text,labels
133,The scent of pine needles in the forest is inv...,0
185,I need help urgently. Can someone please assist?,1
85,A gentle breeze on a summer day is refreshing.,0
121,A sunset picnic is a romantic way to spend an ...,0
106,A genuine compliment can boost someone's confi...,0


In [11]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your own sentiment analysis dataset (replace 'your_dataset.csv' with your file)


# Tokenize text using DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenized_texts = tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    tokenized_texts['input_ids'], df['labels'].tolist(), test_size=0.2, random_state=42
)

# Convert to PyTorch tensors
train_data = TensorDataset(train_texts, torch.tensor(train_labels))
val_data = TensorDataset(val_texts, torch.tensor(val_labels))

# Create DataLoader objects
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False)

# Load pre-trained DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-tune the model
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Evaluate on the validation set
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            outputs = model(inputs, labels=labels)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate accuracy on the validation set
    val_accuracy = accuracy_score(all_labels, all_preds)
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {val_accuracy:.4f}')


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1/3, Validation Accuracy: 1.0000
Epoch 2/3, Validation Accuracy: 1.0000
Epoch 3/3, Validation Accuracy: 1.0000


In [15]:
model.save_pretrained('model/bert_model')
tokenizer.save_pretrained('model/bert_model/')

('model/bert_model/tokenizer_config.json',
 'model/bert_model/special_tokens_map.json',
 'model/bert_model/vocab.txt',
 'model/bert_model/added_tokens.json')

In [2]:
sentences = [
    "Can someone please help me with this problem?",
    "I'm in a difficult situation. Can anyone offer assistance?",
    "I'm feeling lost and need guidance. Can someone help?",
    "Is there anyone who can lend a helping hand?",
    "I don't know what to do. Can someone provide advice?",
    "I'm struggling with a task. Can someone give me a hand?",
    "Help! I'm stuck and can't figure this out.",
    "Could someone please assist me with this issue?",
    "I need help understanding this concept. Can someone explain?",
    "I'm in trouble and need support. Can someone help me?",
    "Is there anyone who can offer some assistance?",
    "Can someone please rescue me from this situation?",
    "I'm reaching out for help. Can someone offer guidance?",
    "I feel overwhelmed. Can someone help me navigate this?",
    "Help! I'm in a bind and need someone to assist me.",
    "Can anyone provide assistance with this problem?",
    "I'm facing a challenge. Can someone lend a hand?",
    "I need help urgently. Can someone please assist me?",
    "Is there someone who can help me through this tough time?",
    "Can someone please help me understand this better?",
    "I'm struggling and need help. Can someone support me?",
    "Could someone offer assistance with this task?",
    "Help me, please! I'm in a difficult situation.",
    "I'm feeling stuck. Can someone give me some advice?",
    "Is there anyone who can provide help right now?",
    "Can someone please help me solve this puzzle?",
    "I'm in need of assistance. Can someone offer help?",
    "I'm facing a dilemma. Can someone provide guidance?",
    "Help! I need someone to rescue me from this problem.",
    "Can anyone offer support with this issue?",
    "I'm feeling overwhelmed. Can someone help me navigate?",
    "I need help urgently. Can someone please assist?",
    "Is there someone who can provide assistance right away?",
    "Can someone please help me with this difficult task?",
    "I'm in a tough spot. Can someone offer a helping hand?",
    "Help me, please! I'm struggling and need assistance.",
    "I'm in a bind and need someone to rescue me. Can you help?",
    "Is there anyone who can provide guidance on this matter?",
    "Can someone please help me figure out this problem?",
    "I'm facing a challenge. Can someone lend their expertise?",
    "I need help urgently. Can someone please assist me?",
    "Can someone offer assistance with this difficult task?",
    "I'm feeling overwhelmed. Can someone help me navigate through this?",
    "Help! I'm in a tough spot and need someone to assist me.",
    "Can anyone provide support with this issue?",
    "I'm feeling stuck. Can someone give me advice on what to do?",
    "Is there anyone who can provide help right now?",
    "Can someone please help me solve this puzzle?",
    "I'm in need of assistance. Can someone offer help?",
    "I'm facing a dilemma. Can someone provide guidance?",
    "Help! I need someone to rescue me from this problem.",
    "Can anyone offer support with this issue?",
    "I'm feeling overwhelmed. Can someone help me navigate?",
    "I need help urgently. Can someone please assist?",
    "Is there someone who can provide assistance right away?",
    "Can someone please help me with this difficult task?",
    "I'm in a tough spot. Can someone offer a helping hand?",
    "Help me, please! I'm struggling and need assistance.",
    "I'm in a bind and need someone to rescue me. Can you help?",
    "Is there anyone who can provide guidance on this matter?",
    "Can someone please help me figure out this problem?",
    "I'm facing a challenge. Can someone lend their expertise?",
    "I need help urgently. Can someone please assist me?",
    "Can someone offer assistance with this difficult task?",
    "I'm feeling overwhelmed. Can someone help me navigate through this?",
    "Help! I'm in a tough spot and need someone to assist me.",
    "Can anyone provide support with this issue?",
    "I'm feeling stuck. Can someone give me advice on what to do?",
    "Is there anyone who can provide help right now?",
    "Can someone please help me solve this puzzle?",
    "I'm in need of assistance. Can someone offer help?",
    "I'm facing a dilemma. Can someone provide guidance?",
    "Help! I need someone to rescue me from this problem.",
    "Can anyone offer support with this issue?",
    "I'm feeling overwhelmed. Can someone help me navigate?",
    "I need help urgently. Can someone please assist?",
    "Is there someone who can provide assistance right away?",
    "Can someone please help me with this difficult task?",
    "I'm in a tough spot. Can someone offer a helping hand?",
    "Help me, please! I'm struggling and need assistance.",
    "I'm in a bind and need someone to rescue me. Can you help?",
    "Is there anyone who can provide guidance on this matter?",
    "Can someone please help me figure out this problem?",
    "I'm facing a challenge. Can someone lend their expertise?",
    "I need help urgently. Can someone please assist me?",
    "Can someone offer assistance with this difficult task?",
    "I'm feeling overwhelmed. Can someone help me navigate through this?",
    "Help! I'm in a tough spot and need someone to assist me.",
    "Can anyone provide support with this issue?",
    "I'm feeling stuck. Can someone give me advice on what to do?",
    "Is there anyone who can provide help right now?",
    "Can someone please help me solve this puzzle?",
    "I'm in need of assistance. Can someone offer help?",
    "I'm facing a dilemma. Can someone provide guidance?",
    "Help! I need someone to rescue me from this problem.",
    "Can anyone offer support with this issue?",
    "I'm feeling overwhelmed. Can someone help me navigate?",
    "I need help urgently. Can someone please assist?",
    "Is there someone who can provide assistance right away?",
    "Can someone please help me with this difficult task?",
    "I'm in a tough spot. Can someone offer a helping hand?",
    "Help me, please! I'm struggling and need assistance.",
    "I'm in a bind and need someone to rescue me. Can you help?",
    "Is there anyone who can provide guidance on this matter?",
    "Can someone please help me figure out this problem?",
    "I'm facing a challenge. Can someone lend their expertise?",
    "I need help urgently. Can someone please assist me?",
    "Can someone offer assistance with this difficult task?",
    "I'm feeling overwhelmed. Can someone help me navigate through this?",
    "Help! I'm in a tough spot and need someone to assist me.",
    "Can anyone provide support with this issue?",
    "I'm feeling stuck. Can someone give me advice on what to do?",
    "Is there anyone who can provide help right now?",
    "Can someone please help me solve this puzzle?",
    "I'm in need of assistance. Can someone offer help?",
    "I'm facing a dilemma. Can someone provide guidance?",
    "Help! I need someone to rescue me from this problem.",
    "Can anyone offer support with this issue?",
    "I'm feeling overwhelmed. Can someone help me navigate?",
    "I need help urgently. Can someone please assist?",
    "Is there someone who can provide assistance right away?",
    "Can someone please help me with this difficult task?",
    "I'm in a tough spot. Can someone offer a helping hand?",
    "Help me, please! I'm struggling and need assistance.",
    "I'm in a bind and need someone to rescue me. Can you help?",
    "Is there anyone who can provide guidance on this matter?",
    "Can someone please help me figure out this problem?",
    "I'm facing a challenge. Can someone lend their expertise?",
    "I need help urgently. Can someone please assist me?",
    "Can someone offer assistance with this difficult task?",
    "I'm feeling overwhelmed. Can someone help me navigate through this?",
    "Help! I'm in a tough spot and need someone to assist me.",
    "Can anyone provide support with this issue?",
    "I'm feeling stuck. Can someone give me advice on what to do?",
    "Is there anyone who can provide help right now?",
    "Can someone please help me solve this puzzle?",
    "I'm in need of assistance. Can someone offer help?",
    "I'm facing a dilemma. Can someone provide guidance?",
    "Help! I need someone to rescue me from this problem.",
    "Can anyone offer support with this issue?",
    "I'm feeling overwhelmed. Can someone help me navigate?",
    "I need help urgently. Can someone please assist?",
    "Is there someone who can provide assistance right away?",
    "Can someone please help me with this difficult task?",
    "I'm in a tough spot. Can someone offer a helping hand?",
    "Help me, please! I'm struggling and need assistance.",
    "I'm in a bind and need someone to rescue me. Can you help?",
    "Is there anyone who can provide guidance on this matter?",
    "Can someone please help me figure out this problem?",
    "I'm facing a challenge. Can someone lend their expertise?",
    "I need help urgently. Can someone please assist me?",
    "Can someone offer assistance with this difficult task?",
    "I'm feeling overwhelmed. Can someone help me navigate through this?",
    "Help! I'm in a tough spot and need someone to assist me.",
    "Can anyone provide support with this issue?",
    "I'm feeling stuck. Can someone give me advice on what to do?",
    "Is there anyone who can provide help right now?",
    "Can someone please help me solve this puzzle?",
    "I'm in need of assistance. Can someone offer help?",
    "I'm facing a dilemma. Can someone provide guidance?",
    "Help! I need someone to rescue me from this problem.",
    "Can anyone offer support with this issue?",
    "I'm feeling overwhelmed. Can someone help me navigate?",
    "I need help urgently. Can someone please assist?",
    "Is there someone who can provide assistance right away?",
    "Can someone please help me with this difficult task?",
    "I'm in a tough spot. Can someone offer a helping hand?",
    "Help me, please! I'm struggling and need assistance.",
    "I'm in a bind and need someone to rescue me. Can you help?",
    "Is there anyone who can provide guidance on this matter?",
    "Can someone please help me figure out this problem?",
    "I'm facing a challenge. Can someone lend their expertise?",
    "I need help urgently. Can someone please assist me?",
    "Can someone offer assistance with this difficult task?",
    "I'm feeling overwhelmed. Can someone help me navigate through this?",
    "Help! I'm in a tough spot and need someone to assist me.",
    "Can anyone provide support with this issue?",
    "I'm feeling stuck. Can someone give me advice on what to do?",
    "Is there anyone who can provide help right now?",
    "Can someone please help me solve this puzzle?",
    "I'm in need of assistance. Can someone offer help?",
    "I'm facing a dilemma. Can someone provide guidance?",
    "Help! I need someone to rescue me from this problem.",
    "Can anyone offer support with this issue?",
    "I'm feeling overwhelmed. Can someone help me navigate?",
    "I need help urgently. Can someone please assist?",
    "Is there someone who can provide assistance right away?",
    "Can someone please help me with this difficult task?",
    "I'm in a tough spot. Can someone offer a helping hand?",
    "Help me, please! I'm struggling and need assistance.",
    "I'm in a bind and need someone to rescue me. Can you help?",
    "Is there anyone who can provide guidance on this matter?",
    "Can someone please help me figure out this problem?",
    "I'm facing a challenge. Can someone lend their expertise?",
    "I need help urgently. Can someone please assist me?",
    "Can someone offer assistance with this difficult task?",
    "I'm feeling overwhelmed. Can someone help me navigate through this?",
    "Help! I'm in a tough spot and need someone to assist me.",
    "Can anyone provide support with this issue?",
    "I'm feeling stuck. Can someone give me advice on what to do?",
    "Is there anyone who can provide help right now?",
    "Can someone please help me solve this puzzle?",
    "I'm in need of assistance. Can someone offer help?",
    "I'm facing a dilemma. Can someone provide guidance?",
    "Help! I need someone to rescue me from this problem.",
    "Can anyone offer support with this issue?",
    "I'm feeling overwhelmed. Can someone help me navigate?",
    "I need help urgently. Can someone please assist?",
    "Is there someone who can provide assistance right away?",
    "Can someone please help me with this difficult task?",
    "I'm in a tough spot. Can someone offer a helping hand?",
    "Help me, please! I'm struggling and need"]
label1= 1
label2=0
random_sentences = [
    "The sunsets here are breathtaking.",
    "She danced like nobody was watching.",
    "Pizza is my favorite comfort food.",
    "The mountain air was crisp and refreshing.",
    "Learning a new language can be challenging but rewarding.",
    "Laughter is the best medicine.",
    "The city skyline sparkled with lights at night.",
    "The old bookstore had a charming atmosphere.",
    "Music has the power to evoke strong emotions.",
    "The smell of freshly baked bread is irresistible.",
    "Raindrops tapped gently on the windowpane.",
    "Painting allows me to express my creativity.",
    "The butterfly fluttered gracefully from flower to flower.",
    "Exploring new cultures broadens your perspective.",
    "Reading a good book is like taking a journey.",
    "A cup of hot tea is soothing on a cold day.",
    "The waves crashed against the shore.",
    "The smell of a barbecue fills the air during summer.",
    "The sound of rain on the roof is a calming lullaby.",
    "Hiking through the forest is a great way to connect with nature.",
    "Ice cream is a delicious treat, especially on a hot day.",
    "Traveling opens your mind to new possibilities.",
    "A warm blanket and a good movie make for a cozy night in.",
    "The smell of coffee in the morning is invigorating.",
    "Gazing at the stars on a clear night is awe-inspiring.",
    "Creating art is a form of self-expression.",
    "Kindness has a ripple effect that can change the world.",
    "The first snowfall transforms the landscape into a winter wonderland.",
    "Singing in the shower is a great stress reliever.",
    "A handwritten letter has a personal touch that emails lack.",
    "The laughter of children is infectious and heartwarming.",
    "Morning walks are a peaceful way to start the day.",
    "A good friend is like a rare gem, precious and valuable.",
    "The smell of a campfire brings back fond memories.",
    "Dancing in the rain is liberating and fun.",
    "A smile can brighten someone's day.",
    "Sunflowers always turn their faces towards the sun.",
    "Freshly squeezed orange juice is a breakfast delight.",
    "A hammock in the backyard is the perfect spot for relaxation.",
    "Chasing fireflies on a summer evening is a nostalgic activity.",
    "The aroma of a barbecue can bring neighbors together.",
    "Solving a puzzle is a satisfying mental challenge.",
    "The beauty of a sunset is a reminder of life's fleeting moments.",
    "A handwritten note can make someone's day.",
    "Meditation brings inner peace and clarity.",
    "A well-balanced diet contributes to overall well-being.",
    "Sitting by the fireplace with a good book is a cozy pastime.",
    "The thrill of a roller coaster ride is exhilarating.",
    "A heartfelt apology can mend relationships.",
    "A positive mindset can overcome challenges.",
    "The rustling of leaves in the wind is a soothing sound.",
    "Taking a deep breath can help reduce stress.",
    "The smell of fresh flowers is invigorating.",
    "A spontaneous road trip can lead to memorable adventures.",
    "Baking cookies fills the house with a delightful aroma.",
    "A handwritten thank-you note shows appreciation.",
    "The feeling of sand between your toes is relaxing at the beach.",
    "A genuine compliment can boost someone's confidence.",
    "Gardening is a therapeutic and rewarding hobby.",
    "The sight of a rainbow after the rain brings joy.",
    "A warm hug can convey more than words.",
    "Watching a sunrise is a peaceful way to start the day.",
    "A hot shower can wash away the stress of the day.",
    "A good night's sleep is essential for well-being.",
    "The excitement of opening a gift is a joyful experience.",
    "A picnic in the park is a simple pleasure.",
    "A handwritten to-do list is a tangible way to stay organized.",
    "The smell of freshly cut grass signals the arrival of summer.",
    "A well-timed joke can lighten the mood.",
    "The sound of waves crashing on the shore is hypnotic.",
    "A warm cup of cocoa is comforting on a cold day.",
    "The aroma of a home-cooked meal is inviting.",
    "A sunset picnic is a romantic way to spend an evening.",
    "A heartfelt compliment can make someone's day.",
    "Climbing a tree is a fun and adventurous activity.",
    "The smell of a new book is intoxicating for avid readers.",
    "A random act of kindness can have a lasting impact.",
    "The satisfaction of completing a puzzle is fulfilling.",
    "A handwritten love letter is a timeless gesture.",
    "The crunch of autumn leaves underfoot is a nostalgic sound.",
    "A well-timed nap can recharge your energy.",
    "The taste of homemade soup warms the soul.",
    "Singing along to your favorite song is a mood booster.",
    "A handwritten note is a thoughtful way to show appreciation.",
    "The scent of pine needles in the forest is invigorating.",
    "A gentle breeze on a summer day is refreshing.",
    "A supportive friend can make challenges easier to face.",
    "The sight of blooming flowers signals the arrival of spring.",
    "A homemade meal cooked with love is the best kind.",
    "A handwritten birthday card adds a personal touch.",
    "The crackling sound of a fireplace is cozy and comforting.",
    "A spontaneous adventure can create lasting memories.",
    "The smell of cinnamon and spices evokes holiday memories.",
    "A good book can transport you to different worlds.",
    "The sight of a shooting star is a magical moment.",
    "A cozy blanket fort is a fun indoor activity.",
    "The feeling of sand between your toes at the beach is therapeutic.",
    "A heartfelt apology can mend relationships.",
    "A positive mindset can overcome challenges.",
    "The rustling of leaves in the wind is a soothing sound.",
    "Taking a deep breath can help reduce stress.",
    "The smell of fresh flowers is invigorating.",
    "A spontaneous road trip can lead to memorable adventures.",
    "Baking cookies fills the house with a delightful aroma.",
    "A handwritten thank-you note shows appreciation.",
    "The feeling of sand between your toes is relaxing at the beach.",
    "A genuine compliment can boost someone's confidence.",
    "Gardening is a therapeutic and rewarding hobby.",
    "The sight of a rainbow after the rain brings joy.",
    "A warm hug can convey more than words.",
    "Watching a sunrise is a peaceful way to start the day.",
    "A hot shower can wash away the stress of the day.",
    "A good night's sleep is essential for well-being.",
    "The excitement of opening a gift is a joyful experience.",
    "A picnic in the park is a simple pleasure.",
    "A handwritten to-do list is a tangible way to stay organized.",
    "The smell of freshly cut grass signals the arrival of summer.",
    "A well-timed joke can lighten the mood.",
    "The sound of waves crashing on the shore is hypnotic.",
    "A warm cup of cocoa is comforting on a cold day.",
    "The aroma of a home-cooked meal is inviting.",
    "A sunset picnic is a romantic way to spend an evening.",
    "A heartfelt compliment can make someone's day.",
    "Climbing a tree is a fun and adventurous activity.",
    "The smell of a new book is intoxicating for avid readers.",
    "A random act of kindness can have a lasting impact.",
    "The satisfaction of completing a puzzle is fulfilling.",
    "A handwritten love letter is a timeless gesture.",
    "The crunch of autumn leaves underfoot is a nostalgic sound.",
    "A well-timed nap can recharge your energy.",
    "The taste of homemade soup warms the soul.",
    "Singing along to your favorite song is a mood booster.",
    "A handwritten note is a thoughtful way to show appreciation.",
    "The scent of pine needles in the forest is invigorating.",
    "A gentle breeze on a summer day is refreshing.",
    "A supportive friend can make challenges easier to face.",
    "The sight of blooming flowers signals the arrival of spring.",
    "A homemade meal cooked with love is the best kind.",
    "A handwritten birthday card adds a personal touch.",
    "The crackling sound of a fireplace is cozy and comforting.",
    "A spontaneous adventure can create lasting memories.",
    "The smell of cinnamon and spices evokes holiday memories.",
    "A good book can transport you to different worlds.",
    "The sight of a shooting star is a magical moment.",
    "A cozy blanket fort is a fun indoor activity.",
]

print(len(random_sentences))



145
